# Домашнее задание. Ансамблирование моделей

- Теперь решаем задачу регрессии - предскажем цены на недвижимость. Использовать датасет https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data (train.csv)
- Данных немного, поэтому необходимо использовать 10-fold кросс-валидацию для оценки качества моделей
- Построить случайный лес, вывести важность признаков
- Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную
- Для валидации модели 2-го уровня использовать отдельный hold-out датасет, как на занятии
- Показать, что использование ансамблей моделей действительно улучшает качество (стекинг vs другие модели сравнивать на hold-out)
- В качестве решения: Jupyter notebook с кодом, комментариями и графиками, ссылка на гитхаб

## 0. Библиотеки

In [40]:
# Для работы с данными
import pandas as pd
import numpy as np

#Для работы с моделями
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

from sklearn.metrics import mean_squared_log_error, mean_squared_error
from sklearn.model_selection import cross_val_score

#Для визуализации
import matplotlib.pyplot as plt
%matplotlib inline

from jupyterthemes import jtplot
jtplot.style()

## 1. Загрузка данных

In [41]:
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

In [42]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

### 1.1 Замена пустых значений и работа с категориальными признаками

In [43]:
# Находим не категориальные признаки
cat_feat_train = list(train_data.dtypes[(train_data.dtypes == object)].index)
cat_feat_test = list(train_data.dtypes[(train_data.dtypes == object)].index)

train_data[cat_feat_train] = train_data[cat_feat_train].fillna('nan')
test_data[cat_feat_test] = test_data[cat_feat_test].fillna('nan')

cat_nunique = train_data[cat_feat_train].nunique()
cat_feat = list(cat_nunique[cat_nunique < 30].index)

# Не категориальные признаки
num_feat = [f for f in train_data if f not in (cat_feat + ['ID', 'SalePrice'])]

In [44]:
# Создаем дамми-переменные для категориальных признаков
dummy_train = pd.get_dummies(train_data[cat_feat], columns=cat_feat)
dummy_test = pd.get_dummies(test_data[cat_feat], columns=cat_feat)

dummy_train_cols = list(set(dummy_train))
dummy_test_cols = list(set(dummy_test))

dummy_train = dummy_train[dummy_train_cols]
dummy_test = dummy_test[dummy_test_cols]

train = pd.concat([train_data[num_feat].fillna(-999), dummy_train], axis=1)
for i in list(set(dummy_test.columns) - set(dummy_train.columns)):
    train[i] = 0

test = pd.concat([test_data[num_feat].fillna(-999), dummy_test], axis=1)
for i in list(set(dummy_train.columns) - set(dummy_test.columns)):
    test[i] = 0

### 1.2 Обучающие и тестовые выборки

In [45]:
X = train.loc[:, train.columns]
y = train_data['SalePrice']

In [46]:
X_kaggle_test = test.loc[:, test.columns]

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

## 2. Модель - Случайный лес. Анализ важности признаков

In [48]:
model_forest = RandomForestRegressor(n_estimators=100, max_depth=5, min_samples_leaf=10, max_features=0.5, n_jobs=-1)

In [49]:
score = cross_val_score(model_forest, X_train, y_train, cv=10)
# Оценка модели с помощью кросс-валидации на 10-fold
score.mean()

0.8226445010329255

In [50]:
# Зафитим модель и посчитаем предсказания для тестовой выборки
model_forest.fit(X,y)
forest_predictions = model_forest.predict(X_test)

In [51]:
# Оценка качества модели исходя из метрики, обозначенной на Kaggle - Mean Squared Log Error
mean_squared_log_error(y_test, forest_predictions)

0.02533744194969466

In [52]:
kaggle_preditions = model_forest.predict(X_kaggle_test)

In [53]:
# Записываем данные для отправки на Kaggle
result = X_kaggle_test.loc[:, ['Id']]
result['SalePrice'] = kaggle_preditions
result.to_csv('forest_results.csv', index=False)

Выполнили сабмит на Kaggle, после чего получила не самый лучший результат:

<img src='kaggle_forest.png'>

При этом оценка на тестовых данных из выборки и оценка на Kaggle отличается весьма сильно, что может свидетельствовать о переобучении модели

In [54]:
#Важность признаков для модели
Columns = pd.DataFrame(X.columns, columns=['column'])
Columns['importance'] = pd.Series(model_forest.feature_importances_)
#Выбираем только значимые признаки для модели и сортируем их в порядке значимости
Columns[(Columns['importance']>0)].head(25).sort_values('importance', ascending=False).style.bar()

,column,importance
4,OverallQual,0.464497
26,GarageCars,0.118014
16,GrLivArea,0.101252
12,TotalBsmtSF,0.0416176
13,1stFlrSF,0.033762
6,YearBuilt,0.0326264
9,BsmtFinSF1,0.0224639
27,GarageArea,0.0221748
19,FullBath,0.0176019
3,LotArea,0.0111745


In [55]:
# Выберем только наиболее значимые признаки, для которых коэффициенты >0.00
columns = Columns[(Columns['importance']>0.001)].column.unique()
columns

array(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'YearBuilt',
       'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF',
       'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'BsmtFullBath',
       'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'KitchenQual_Gd', 'BsmtQual_Ex', 'CentralAir_N',
       'FireplaceQu_nan', 'ExterQual_Gd', 'KitchenQual_TA',
       'KitchenQual_Ex', 'GarageType_Attchd', 'ExterQual_TA',
       'CentralAir_Y', 'ExterQual_Ex'], dtype=object)

###### Создадим новые обучающие выборки для стэкинга моделей

In [56]:
X = train.loc[:, columns]
y = train_data['SalePrice']

In [57]:
X_kaggle_test = test.loc[:, columns]

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

## 3. Стекинг из трех моделей

Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную
Для валидации модели 2-го уровня использовать отдельный hold-out датасет, как на занятии

In [59]:
#Модели для получения мета-признаков
# from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

#Инструмент для разбиения на фолды
from sklearn.model_selection import StratifiedKFold

In [60]:
# Три модели для стекинга
stack_models = [ # Случайный лес
                 DecisionTreeRegressor(),
                 # Логистическая регрессия
                 SVR(kernel='linear', gamma='scale'),
                 # Линейная регрессия
                 LinearRegression()
                ]

In [61]:
# Функция для получения мета-признаков
def get_meta_features(clf, X_train, y_train, stack_cv):
    meta_train = np.zeros_like(y_train, dtype=float)
    meta_test = np.zeros_like(y_test, dtype=float)
    scoring = []
    
    for i, (train_ind, test_ind) in enumerate(stack_cv.split(X_train, y_train)):
        clf.fit(X_train.iloc[train_ind], y_train.iloc[train_ind])
        meta_train[test_ind] = clf.predict(X_train.iloc[test_ind])
        # meta_test += clf.predict(X_test)
        scoring += [clf.score(X_train.iloc[test_ind], y_train.iloc[test_ind])]
        
    print('Scoring: {0}'.format(scoring))
    
    return meta_train #, meta_test / stack_cv.n_splits

In [62]:
# Разбиваем на фолды
stack_cv = StratifiedKFold(n_splits=4, random_state=123)

In [63]:
# Получаем метапризнаки для мета-модели
meta_train = []
meta_test = []
col_names = []

# Создание мета-признаков
for i in stack_models:
    print("Обучение и создание признаков для модели {0}".format(i.__class__.__name__))
    meta_train_new = get_meta_features(i, X_train, y_train, stack_cv) #, meta_test_new 
    meta_train.append(meta_train_new)
    #meta_test.append(meta_test_new)
    col_names.append(i.__class__.__name__)
    

Обучение и создание признаков для модели DecisionTreeRegressor
Scoring: [0.6876942021764868, 0.3667378966813518, 0.6012193461918031, 0.5050786220087387]
Обучение и создание признаков для модели SVR


/Users/akirichenko/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)
/Users/akirichenko/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)


Scoring: [0.68630778736434, 0.2692580663643327, 0.7692068957746431, 0.7278521940712271]
Обучение и создание признаков для модели LinearRegression


/Users/akirichenko/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)


Scoring: [0.683074983173455, 0.4133316380711316, 0.7864628222173078, 0.791928076314945]


In [64]:
# Собираем полученные мета-признаки, для обучения мета-модели
X_meta_train = pd.DataFrame({col_names[i]: meta_train[i] for i in range(len(col_names))})

## Обучаем главную модель и получаем результат

In [65]:
# Обучаем главную мета-модель
meta_model = RandomForestRegressor(n_estimators=100, max_depth=5, min_samples_leaf=10, max_features=0.5, n_jobs=-1)
meta_model.fit(X_meta_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features=0.5, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=10, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [66]:
# Обучаем стекинговые модели на всей обучающей выборке и получаем мета-признаки для hold-out набора:
meta_test = []
for i in stack_models:
    i.fit(X_train, y_train)
    print('Оценка модели {0} : {1}'.format(i.__class__.__name__ , i.score(X_test, y_test)))
    meta_test.append(i.predict(X_test))

X_meta_test = pd.DataFrame({col_names[i]: meta_test[i] for i in range(len(col_names))})

Оценка модели DecisionTreeRegressor : 0.8762461362588587
Оценка модели SVR : 0.792329014982595
Оценка модели LinearRegression : 0.8532257852181153


In [67]:
# Оцениваем мета-модель на отложенной hold_out-выборке
meta_model.score(X_meta_test, y_test)

0.7801491737032363

In [68]:
predictions = meta_model.predict(X_meta_test)

In [69]:
# Оценка качества модели исходя из метрики, обозначенной на Kaggle - Mean Squared Log Error
mean_squared_log_error(y_test, predictions)

0.026465833163378534

In [70]:
# Сделаем оценку для тестовой выборки на Kaggle
meta_kaggle_test = []
for i in stack_models:
    meta_kaggle_test.append(i.predict(X_kaggle_test))

X_meta_kaggle_test = pd.DataFrame({col_names[i]: meta_kaggle_test[i] for i in range(len(col_names))})

In [71]:
kaggle_predictions = meta_model.predict(X_meta_kaggle_test)
# Записываем данные для отправки на Kaggle
result = test.loc[:, ['Id']]
result['SalePrice'] = kaggle_predictions
result.to_csv('staking_results.csv', index=False)

Загружаем результаты на Kaggle :

<img src='kaggle_staking.png'>

Результат улучшился.